# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,candido mota,-22.7464,-50.3869,19.51,93,95,3.28,BR,1702266663
1,1,lihue,21.9789,-159.3672,24.77,84,75,4.12,US,1702266530
2,2,bubaque,11.2833,-15.8333,28.09,59,100,4.57,GW,1702266663
3,3,ilulissat,69.2167,-51.1000,-1.99,64,100,9.26,GL,1702266663
4,4,whitehorse,60.7161,-135.0538,-10.66,84,100,8.23,CA,1702266664


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display

humidity = city_data_df["Humidity"].astype(int)

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    color="City",
    alpha=.75
)

# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_df = city_data_df.loc[(city_data_df["Max Temp"] < 27) & (city_data_df["Max Temp"] > 21)]
city_data_df = city_data_df.loc[(city_data_df["Wind Speed"] < 4.5)]
city_data_df = city_data_df.loc[(city_data_df["Cloudiness"] == 0)]

# Drop any rows with null values
city_data_df = city_data_df.dropna()

# Display sample data
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
18,18,adamstown,-25.0660,-130.1015,21.31,57,0,2.84,PN,1702266667
20,20,margaret river,-33.9500,115.0667,25.10,35,0,3.23,AU,1702266667
49,49,cabo san lucas,22.8909,-109.9124,24.31,63,0,3.09,MX,1702266029
96,96,tura,25.5198,90.2201,21.47,49,0,0.37,IN,1702266689
219,219,jinxiang,27.4327,120.6062,21.65,69,0,1.08,CN,1702266710
364,364,jinghong,21.9910,100.7341,26.22,53,0,1.01,CN,1702266731
467,467,san jose de jachal,-30.2406,-68.7469,21.44,43,0,2.71,AR,1702266750


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
city = city_data_df["City"]
country = city_data_df["Country"]
lat = city_data_df["Lat"]
lng = city_data_df["Lng"]
humidity = city_data_df["Humidity"]

hotel_df = city_data_df[{"City": city,
                        "Country": country,
                        "Lat": lat,
                        "Lng": lng,
                        "Humidity": humidity}]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

C:\Users\carlo\AppData\Local\Temp\ipykernel_14688\2468048700.py:8: FutureWarning: Passing a dict as an indexer is deprecated and will raise in a future version. Use a list instead.
  hotel_df = city_data_df[{"City": city,
C:\Users\carlo\AppData\Local\Temp\ipykernel_14688\2468048700.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = ""


,City,Country,Lat,Lng,Humidity,Hotel Name
18,adamstown,PN,-25.0660,-130.1015,57,
20,margaret river,AU,-33.9500,115.0667,35,
49,cabo san lucas,MX,22.8909,-109.9124,63,
96,tura,IN,25.5198,90.2201,49,
219,jinxiang,CN,27.4327,120.6062,69,
364,jinghong,CN,21.9910,100.7341,53,
467,san jose de jachal,AR,-30.2406,-68.7469,43,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lon = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
adamstown - nearest hotel: No hotel found
margaret river - nearest hotel: Margaret River Hotel
cabo san lucas - nearest hotel: Comfort Rooms
tura - nearest hotel: No hotel found
jinxiang - nearest hotel: No hotel found
jinghong - nearest hotel: โรงแรมสืเบา
san jose de jachal - nearest hotel: San Martin


,City,Country,Lat,Lng,Humidity,Hotel Name
18,adamstown,PN,-25.0660,-130.1015,57,No hotel found
20,margaret river,AU,-33.9500,115.0667,35,Margaret River Hotel
49,cabo san lucas,MX,22.8909,-109.9124,63,Comfort Rooms
96,tura,IN,25.5198,90.2201,49,No hotel found
219,jinxiang,CN,27.4327,120.6062,69,No hotel found
364,jinghong,CN,21.9910,100.7341,53,โรงแรมสืเบา
467,san jose de jachal,AR,-30.2406,-68.7469,43,San Martin


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [8]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    color="City",
    alpha=.75,
    hover_cols=["Hotel Name", "Country"]
)
# Display the map
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)